## Prerequisites

### Google Colab Environment Checking

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

### MacOS Environment Checking

In this code, platform.system() returns the name of the operating system dependent module imported. The returned value is 'Darwin' for MacOS, 'Linux' for Linux, 'Windows' for Windows and so on. If the returned value is 'Darwin', it means you are using MacOS.

In [3]:
import platform
import distro

if platform.system() == 'Darwin':
    IN_MACOS = True
else:
    IN_MACOS = False
    
if platform.system() == 'Linux':
    distro_name = distro.id()
    if 'debian' in distro_name.lower() or 'ubuntu' in distro_name.lower():
        IN_DEBIAN = True
    else:
        IN_DEBIAN = False
else:
    IN_DEBIAN = False

### Check if MPI installed in OS

Use the mpirun command to see if MPI is up and running.

In [4]:
import subprocess

def is_mpi_installed():
    try:
        if IN_MACOS:
          subprocess.check_output(["/usr/local/bin/mpirun", "--version"])
        else:
          subprocess.check_output(["mpirun", "--version"])
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

mpi_installed = is_mpi_installed()
if not mpi_installed:
    print("[FATAL] MPI is not installed")

### Check if NVIDIA CUDA toolkit installed

Use the nvcc command to see if MPI is up and running.

In [5]:
import subprocess

def is_cuda_installed():
    try:
        subprocess.check_output(["nvcc", "--version"])
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

cuda_installed = is_cuda_installed()
if not cuda_installed:
    print("[FATAL] CUDA is not installed")

### Install MPI and CUDA if not installed

**Reminder**: Because latest Macbook does not bundle with NVIDIA CUDA compatible GPU and CUDA toolkits since at least CUDA 4.0 have not supported an ability to run cuda code without a GPU, this program cannot support MacOS environment.

If mpi_installed of the above result show False, please install openmpi binary and library based on your platform.

In Ubuntu you can install Open MPI as follow

```bash
sudo apt update
sudo apt install openmpi-bin
sudo apt install libopenmpi-dev
```

The following code will install Open MPI in Google Colab

In [6]:
import os

if IN_COLAB and not mpi_installed:
    !apt update
    !apt install openmpi-bin
    !apt install libopenmpi-dev
    

if cuda_installed show False, please install NVIDIA CUDA toolkit in your platform

In Ubuntu you can install CUDA as follow

```bash
sudo apt update
sudo apt install -y gpupg2
wget https://developer.download.nvidia.com/compute/cuda/repos/debian10/x86_64/cuda-repo-debian10_10.2.89-1_amd64.deb
sudo dpkg -i cuda-repo-debian10_10.2.89-1_amd64.deb
sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/debian10/x86_64/7fa2af80.pub
sudo apt update
sudo apt-get install cuda
```

Under Google Colab, cuda is bundled with T4 GPU instance.

## Environment Setup

### Kaggle Authenticiation

In this notebook, we will download a dataset from Kaggle. Before beginning the download process, it is necessary to ensure an account on Kaggle available. If you do not wish to sign in and would rather bypass the login prompt by uploading your kaggle.json file directly instead, then obtain it from your account settings page and save it either in the project root directory or content directory of Google Colab before starting this notebook. This way, you can quickly access any datasets without needing to log into Kaggle every time!

### Install PyPi packages

Installing PyPi packages is an essential step in this notebook. Among the mandatory packages, mpi4py and opendatasets provide crucial functionalities for data manipulation, distributed computing, and accessing large datasets. While Google Colab offers the convenience of bundled packages such as numpy, matplotlib, pandas, and seaborn, these packages still need to be installed separately in a local environment.

In [12]:
%pip install --upgrade pip
%pip install mpi4py
%pip install opendatasets
%pip install yfinance

if cuda_installed:
    %pip install pycuda

if not IN_COLAB:
    %pip install numpy
    %pip install matplotlib
    %pip install pandas
    %pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

### Import required packages

In [13]:
import numpy as np
import opendatasets as od
import yfinance as yf
import csv
import os
import pandas as pd
from datetime import datetime

if cuda_installed:
    import pycuda.autoinit
    import pycuda.driver as cuda
    from pycuda.compiler import SourceModule

## S&P 500 Constituents Dataset Download

I will first need to download S&P 500 constituents from my Kaggle repository

In [14]:
od.download("https://www.kaggle.com/datasets/reidlai/s-and-p-500-constituents")

Skipping, found downloaded files in "./s-and-p-500-constituents" (use force=True to force download)


## Stock Price History Download

In [17]:
class Row:
    def __init__(self, timestamp, open, high, low, close, adjclose, volume):
        self.timestamp = timestamp
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.adjclose = adjclose
        self.volume = volume

def get_stock_price_history_quotes(stock_symbol, start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%S%z")
    end_date = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%S%z")

    try:
        data = yf.download(stock_symbol, start=start_date, end=end_date)
    except Exception as e:
        logging.error(f"Symbol not found: {stock_symbol}")
        return []

    quotes = []
    for index, row in data.iterrows():
        quote = Row(index, row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume'])
        quotes.append(quote)

    quotes.sort(key=lambda x: x.timestamp)
    
    # convert quotes into dataframe
    quotes_df = pd.DataFrame([vars(quote) for quote in quotes])
    # add symbol column
    quotes_df['symbol'] = stock_symbol
    return quotes_df

## Technical Analysis

### CPU based technical indicator funtions

In [15]:
def ema(days, values):
    alpha = 2 / (days + 1)
    ema_values = [values[0]]  # start with the first value
    for value in values[1:]:
        ema_values.append(alpha * value + (1 - alpha) * ema_values[-1])
    return ema_values[-1]

def rsi(days, values):
    gains = []
    losses = []
    for i in range(1, len(values)):
        change = values[i] - values[i - 1]
        if change > 0:
            gains.append(change)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(-change)
    avg_gain = sum(gains[:days]) / days
    avg_loss = sum(losses[:days]) / days
    rs = avg_gain / avg_loss if avg_loss != 0 else 0
    rsi_value = 100 - (100 / (1 + rs))
    return rsi_value

def macd(values, short_period=12, long_period=26, signal_period=9):
    ema_short = exponential_moving_average(short_period, values)
    ema_long = exponential_moving_average(long_period, values)
    macd_line = np.array(ema_short) - np.array(ema_long)
    signal_line = exponential_moving_average(signal_period, macd_line.tolist())
    return macd_line, signal_line

### GPU based technical indicator funtions

In [ ]:
if cuda_installed:
    
    mod = SourceModule("""
    __global__ void ema(float *values, float *ema_values, int days, int n, int m) {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        if (idx < n) {
            float alpha = 2.0f / (days + 1);
            for (int j = 0; j < m; j++) {
                if (j == 0) {
                    ema_values[idx * m] = values[idx * m];  // start with the first value
                } else {
                    ema_values[idx * m + j] = alpha * values[idx * m + j] + (1 - alpha) * ema_values[idx * m + j - 1];
                }
            }
        }
    }

    __global__ void compute_gains_losses(float *values, float *gains, float *losses, int n) {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        if (idx < n) {
            float change = values[idx] - values[idx - 1];
            gains[idx] = change > 0 ? change : 0;
            losses[idx] = change < 0 ? -change : 0;
        }
    }

    __global__ void macd(float *values, float *macd_values, float *signal_values, int short_period, int long_period, int signal_period, int n, int m) {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        if (idx < n) {
            float alpha_short = 2.0f / (short_period + 1);
            float alpha_long = 2.0f / (long_period + 1);
            float alpha_signal = 2.0f / (signal_period + 1);
            float ema_short = 0;
            float ema_long = 0;
            float ema_signal = 0;
            for (int j = 0; j < m; j++) {
                if (j < short_period) {
                    ema_short = alpha_short * values[idx * m + j] + (1 - alpha_short) * ema_short;
                }
                if (j < long_period) {
                    ema_long = alpha_long * values[idx * m + j] + (1 - alpha_long) * ema_long;
                }
                float macd = ema_short - ema_long;
                if (j < signal_period) {
                    ema_signal = alpha_signal * macd + (1 - alpha_signal) * ema_signal;
                }
                macd_values[idx * m + j] = macd;
                signal_values[idx * m + j] = ema_signal;
            }
        }
    }
    """)

    def ema_gpu(values, days):
        n, m = values.shape
        ema_values = np.empty_like(values)
        block_size = 256
        grid_size = (n + block_size - 1) // block_size
        func = mod.get_function("ema")
        func(cuda.In(values), cuda.Out(ema_values), np.int32(days), np.int32(n), np.int32(m), block=(block_size,1,1), grid=(grid_size,1))
        return ema_values

    def rsi_gpu(days, values):
        n = len(values)
        gains = np.empty_like(values)
        losses = np.empty_like(values)
        block_size = 256
        grid_size = (n + block_size - 1) // block_size
        func = mod.get_function("compute_gains_losses")
        func(cuda.In(values), cuda.Out(gains), cuda.Out(losses), np.int32(n), block=(block_size,1,1), grid=(grid_size,1))

        avg_gain = np.sum(gains[:days]) / days
        avg_loss = np.sum(losses[:days]) / days
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        return rsi_value

    def macd_gpu(values, short_period=12, long_period=26, signal_period=9):
        n, m = values.shape
        macd_values = np.empty_like(values)
        signal_values = np.empty_like(values)
        block_size = 256
        grid_size = (n + block_size - 1) // block_size
        func = mod.get_function("macd")
        func(cuda.In(values), cuda.Out(macd_values), cuda.Out(signal_values), np.int32(short_period), np.int32(long_period), np.int32(signal_period), np.int32(n), np.int32(m), block=(block_size,1,1), grid=(grid_size,1))
        return macd_values, signal_values

## Core Main Program

In [16]:
def main():
    print("abc")
  
if __name__ == "__main__":
    main()

abc


## Data Visualization

## Performance Analysis